# Advent of Code 2020 - Day 24

I used a graph in my first attempt, but it quickly became a nightmare since it required adding edges between all neighbors. This solution takes advantage of the regularity of the [hexagonal tiling](https://en.wikipedia.org/wiki/Hexagonal_tiling). I keep track of tiles in a hash map with keys being a coordinate $(x, y)$. I assume a shift to the right with each decreasing $y$ e.g., the tile at the location $(0, 0$)'s neighbor to the southeast is in location $(0, -1)$.

In [1]:
data = []
with open('inputs_day_24.txt', 'r') as f:
  for line in f:
    data.append(line.strip())

print(len(data))

430


In [2]:
# A little clunky, but it works
def Parse_Instruction(line):
  instructions = []
  cursor = 0
  while(cursor < len(line)):
    instruction = ''
    #print(line, cursor)
    if(line[cursor] == 's'): # se, sw
      instruction = 's'
      cursor += 1
      if(cursor < len(line)):
        if(line[cursor] == 'w' or line[cursor] == 'e'):
          instruction += line[cursor]
          cursor += 1

    elif(line[cursor] == 'n'): # ne, nw
      instruction = 'n'
      cursor += 1
      if(cursor < len(line)):
        if(line[cursor] == 'w' or line[cursor] == 'e'):
          instruction += line[cursor]
          cursor += 1

    elif(line[cursor] == 'e'): # e
      instruction = 'e'
      cursor += 1

    elif(line[cursor] == 'w'): # w
      instruction = 'w'
      cursor += 1

    instructions.append(instruction)
  return instructions

instructions = [Parse_Instruction(line) for line in data]

In [3]:
instruction = instructions[0]

Center = (0, 0)
Tile_Map = {Center : 'White'} # Assume we shift left for decreasing y

for i, instruction in enumerate(instructions):
  current = Center
  for j, step in enumerate(instruction):
    new = None
    if(step == 'e'):
      new = (current[0] + 1, current[1])
    if(step == 'se'): # by default
      new = (current[0], current[1] - 1)
    if(step == 'sw'): # shift right to compensate for default shift to left
      new = (current[0] - 1, current[1] - 1)
    if(step == 'w'):
      new = (current[0] - 1, current[1])
    if(step == 'nw'):
      new = (current[0], current[1] + 1)
    if(step == 'ne'): 
      new = (current[0] + 1, current[1] + 1)

    # Add new tile
    if(not new in Tile_Map.keys()):
        Tile_Map[new] = 'White'

    # Update current
    current = new
    
    # Flip one found the tile
    if(j == len(instruction) - 1):
      if(Tile_Map[current] == 'White'): Tile_Map[current] = 'Black'
      else: Tile_Map[current] = 'White'
    
print(len([color for color in list(Tile_Map.values()) if color == 'Black']))

388


## Part 2

In [4]:
def get_tile_neighbors(tile):
  e = (tile[0] + 1, tile[1])
  se = (tile[0], tile[1] - 1)
  sw = (tile[0] - 1, tile[1] - 1)
  w = (tile[0] - 1, tile[1])
  nw = (tile[0], tile[1] + 1)
  ne = (tile[0] + 1, tile[1] + 1)

  return e, se, sw, w, nw, ne


In [5]:
import copy

Tile_Map_Prev = copy.deepcopy(Tile_Map)

for day in range(100):
  #print(Tile_Map_Prev)
  Tile_Map_New = copy.deepcopy(Tile_Map_Prev)

  # Black to white
  for tile in Tile_Map_Prev:
    tile_neighbors = get_tile_neighbors(tile)
    black_counter = 0
    for tile_neighbor in tile_neighbors:
      if(tile_neighbor in Tile_Map_Prev.keys()):
        if(Tile_Map_Prev[tile_neighbor] == 'Black'): 
          black_counter += 1
      #else:
      #  Tile_Map_Prev[tile_neighbor] = 'White' # Create a new one

    if(Tile_Map_Prev[tile] == 'Black'):
      if(black_counter == 0 or black_counter > 2):
        Tile_Map_New[tile] = 'White'


  # White to black is a bit more complicated. 
  # Need to grab black tiles, for each, find its white neighbors,
  # for each white neighbor, check if it has exactly two black neighbors
  # if it does, make the white neighbor black
  for tile in Tile_Map_Prev:
    if(Tile_Map_Prev[tile] == 'Black'):
      black_tile_neighbors = get_tile_neighbors(tile)
      for black_tile_neighbor in black_tile_neighbors:
        is_white = False
        if(black_tile_neighbor in Tile_Map_Prev):
          if(Tile_Map_Prev[black_tile_neighbor] == 'White'):
            is_white = True
        else: 
          is_white = True # If not in list, must be white
        
        if(is_white):
          black_tile_white_neighbor_neighbors = get_tile_neighbors(black_tile_neighbor)
          black_counter = 0
          for black_tile_white_neighbor_neighbor in black_tile_white_neighbor_neighbors:
            if(black_tile_white_neighbor_neighbor in Tile_Map_Prev.keys()):
              if(Tile_Map_Prev[black_tile_white_neighbor_neighbor] == 'Black'): 
                black_counter += 1

          if(black_counter == 2):
            Tile_Map_New[black_tile_neighbor] = 'Black'



  #print(Tile_Map_New)
  num_black_prev = len([color for color in list(Tile_Map_Prev.values()) if color == 'Black'])
  num_black_new = len([color for color in list(Tile_Map_New.values()) if color == 'Black'])
  print('Day {}: {} to {}'.format(day + 1, num_black_prev, num_black_new))

  Tile_Map_Prev = Tile_Map_New



Day 1: 388 to 294
Day 2: 294 to 329
Day 3: 329 to 357
Day 4: 357 to 387
Day 5: 387 to 356
Day 6: 356 to 395
Day 7: 395 to 387
Day 8: 387 to 461
Day 9: 461 to 468
Day 10: 468 to 462
Day 11: 462 to 495
Day 12: 495 to 511
Day 13: 511 to 535
Day 14: 535 to 577
Day 15: 577 to 567
Day 16: 567 to 619
Day 17: 619 to 609
Day 18: 609 to 671
Day 19: 671 to 677
Day 20: 677 to 682
Day 21: 682 to 720
Day 22: 720 to 762
Day 23: 762 to 821
Day 24: 821 to 772
Day 25: 772 to 813
Day 26: 813 to 858
Day 27: 858 to 856
Day 28: 856 to 896
Day 29: 896 to 907
Day 30: 907 to 902
Day 31: 902 to 1033
Day 32: 1033 to 1051
Day 33: 1051 to 1061
Day 34: 1061 to 1011
Day 35: 1011 to 1085
Day 36: 1085 to 1125
Day 37: 1125 to 1225
Day 38: 1225 to 1222
Day 39: 1222 to 1259
Day 40: 1259 to 1268
Day 41: 1268 to 1321
Day 42: 1321 to 1341
Day 43: 1341 to 1341
Day 44: 1341 to 1429
Day 45: 1429 to 1417
Day 46: 1417 to 1509
Day 47: 1509 to 1519
Day 48: 1519 to 1651
Day 49: 1651 to 1588
Day 50: 1588 to 1643
Day 51: 1643 to 1585